In [1]:
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.WARN)
import pickle
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
import os
from tensorflow.python.client import device_lib

In [2]:
f = open('../Glove/word_embedding_glove', 'rb')
word_embedding = pickle.load(f)
f.close()
word_embedding = word_embedding[: len(word_embedding)-1]

f = open('../Glove/vocab_glove', 'rb')
vocab = pickle.load(f)
f.close()

word2id = dict((w, i) for i,w in enumerate(vocab))
id2word = dict((i, w) for i,w in enumerate(vocab))

unknown_token = "UNKNOWN_TOKEN"

f = open("train.pickle", 'rb')
full_data = pickle.load(f)
f.close()

In [6]:
# Model Description
sense_word = 'hard'
model_name = 'model-p-2'
model_dir = 'output/' + sense_word + '/' + model_name
save_dir = os.path.join(model_dir, "save/")
log_dir = os.path.join(model_dir, "log")

if not os.path.exists(model_dir):
    os.mkdir(model_dir)
if not os.path.exists(save_dir):
    os.mkdir(save_dir)
if not os.path.exists(log_dir):
    os.mkdir(log_dir)

In [7]:
# Parameters
mode = 'train'
num_senses = 3
batch_size = 64
vocab_size = len(vocab)
unk_vocab_size = 1
word_emb_size = len(word_embedding[0])
max_sent_size = 200
hidden_size = 100
keep_prob = 0.5
l2_lambda = 0.001
init_lr = 0.01
decay_steps = 500
decay_rate = 0.96
clip_norm = 1
clipping = True

In [8]:
# MODEL
x = tf.placeholder('int32', [batch_size, max_sent_size], name="x")
y = tf.placeholder('int32', [batch_size], name="y")
x_mask  = tf.placeholder('bool', [batch_size, max_sent_size], name='x_mask') 
is_train = tf.placeholder('bool', [], name='is_train')
word_emb_mat = tf.placeholder('float', [None, word_emb_size], name='emb_mat')
input_keep_prob = tf.cond(is_train,lambda:keep_prob, lambda:tf.constant(1.0))
x_len = tf.reduce_sum(tf.cast(x_mask, 'int32'), 1)

with tf.name_scope("word_embedding"):
    if mode == 'train':
        unk_word_emb_mat = tf.get_variable("word_emb_mat", dtype='float', shape=[unk_vocab_size, word_emb_size], initializer=tf.contrib.layers.xavier_initializer(uniform=False, seed=0, dtype=tf.float32))
    else:
        unk_word_emb_mat = tf.get_variable("word_emb_mat", shape=[unk_vocab_size, word_emb_size], dtype='float')
        
    final_word_emb_mat = tf.concat([word_emb_mat, unk_word_emb_mat], 0)
    Wx = tf.nn.embedding_lookup(final_word_emb_mat, x)  

with tf.variable_scope("lstm1"):
    cell_fw1 = tf.contrib.rnn.BasicLSTMCell(hidden_size,state_is_tuple=True)
    cell_bw1 = tf.contrib.rnn.BasicLSTMCell(hidden_size,state_is_tuple=True)

    d_cell_fw1 = tf.contrib.rnn.DropoutWrapper(cell_fw1, input_keep_prob=input_keep_prob)
    d_cell_bw1 = tf.contrib.rnn.DropoutWrapper(cell_bw1, input_keep_prob=input_keep_prob)
    
    (fw_h1, bw_h1), _ = tf.nn.bidirectional_dynamic_rnn(d_cell_fw1, d_cell_bw1, Wx, sequence_length=x_len, dtype='float', scope='lstm1')
    h1 = tf.concat([fw_h1, bw_h1], 2)
    
with tf.variable_scope("lstm2"):
    cell_fw2 = tf.contrib.rnn.BasicLSTMCell(hidden_size,state_is_tuple=True)
    cell_bw2 = tf.contrib.rnn.BasicLSTMCell(hidden_size,state_is_tuple=True)

    d_cell_fw2 = tf.contrib.rnn.DropoutWrapper(cell_fw2, input_keep_prob=input_keep_prob)
    d_cell_bw2 = tf.contrib.rnn.DropoutWrapper(cell_bw2, input_keep_prob=input_keep_prob)
    
    (fw_h2, bw_h2), _ = tf.nn.bidirectional_dynamic_rnn(d_cell_fw2, d_cell_bw2, h1, sequence_length=x_len, dtype='float', scope='lstm2')
    h = tf.concat([fw_h2, bw_h2], 2)

def attention(input_x, input_mask, W_att):
    h_masked = tf.boolean_mask(input_x, input_mask)
    h_tanh = tf.tanh(h_masked)
    u = tf.matmul(h_tanh, W_att)
    a = tf.nn.softmax(u)
    c = tf.reduce_sum(tf.multiply(h_tanh, a), 0)  
    return c

with tf.variable_scope("attention"):
    W_att = tf.Variable(tf.truncated_normal([2*hidden_size, 1], mean=0.0, stddev=0.1, seed=0), name="W_att")
    c = tf.expand_dims(attention(h[0], x_mask[0], W_att), 0)
    for i in range(1, batch_size):
        c = tf.concat([c, tf.expand_dims(attention(h[i], x_mask[i], W_att), 0)], 0)
        
with tf.variable_scope("softmax_layer"):
    W = tf.Variable(tf.truncated_normal([2*hidden_size, num_senses], mean=0.0, stddev=0.1, seed=0), name="W")
    b = tf.Variable(tf.zeros([num_senses]), name="b")
    drop_c = tf.nn.dropout(c, input_keep_prob)
    logits = tf.matmul(drop_c, W) + b
    predictions = tf.argmax(logits, 1)

loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y))
global_step = tf.Variable(0, trainable=False, name="global_step")

learning_rate = tf.train.exponential_decay(init_lr, global_step, decay_steps, decay_rate, staircase=True)

tv_all = tf.trainable_variables()
tv_regu =[]
for t in tv_all:
    if t.name.find('b:')==-1:
        tv_regu.append(t)
        
# l2 Loss
l2_loss = l2_lambda * tf.reduce_sum([ tf.nn.l2_loss(v) for v in tv_regu ])

total_loss = loss + l2_loss

# Optimizer for loss
optimizer = tf.train.AdamOptimizer(learning_rate)

# Gradients and Variables for Loss
grads_vars = optimizer.compute_gradients(total_loss)

# Clipping of Gradients
clipped_grads = grads_vars
if(clipping == True):
    clipped_grads = [(tf.clip_by_norm(grad, clip_norm), var) for grad, var in clipped_grads]

# Training Optimizer for Total Loss
train_op = optimizer.apply_gradients(clipped_grads, global_step=global_step)

# Summaries
var_summaries = []
for v in tv_all:
    var_summary = tf.summary.histogram("{}/var".format(v.name), v)
    var_summaries.append(var_summary)

var_summaries_merged = tf.summary.merge(var_summaries)

loss_summary = tf.summary.scalar("loss", loss)
total_loss_summary = tf.summary.scalar("total_loss", total_loss)
summary = tf.summary.merge_all()

In [9]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"
sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True))
sess.run(tf.global_variables_initializer())                          # For initializing all the variables
saver = tf.train.Saver()                                             # For Saving the model
summary_writer = tf.summary.FileWriter(log_dir, sess.graph)          # For writing Summaries

In [10]:
# Splitting
data_x = full_data[sense_word][0]
data_y = full_data[sense_word][2]
x_train, x_test, y_train, y_test = train_test_split(data_x, data_y, train_size=0.8, shuffle=True, stratify=data_y, random_state=0)

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, train_size=0.9, shuffle=True, stratify=y_train, random_state=0)

/users/btech/aviraj/cs771/lib/python3.5/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [11]:
def data_prepare(x):
    num_examples = len(x)

    xx = np.zeros([num_examples, max_sent_size], dtype=int)
    xx_mask = np.zeros([num_examples, max_sent_size], dtype=bool)

    for j in range(num_examples):
        for i in range(max_sent_size):
            if(i>=len(x[j])):
                break
            w = x[j][i]
            xx[j][i] = word2id[w] if w in word2id else word2id['UNKNOWN_TOKEN']
            xx_mask[j][i] = True
            
    return xx, xx_mask

def eval_score(yy, pred):
    num_batches = int(len(yy)/batch_size)
    f1 = f1_score(yy[:batch_size*num_batches], pred, average='macro')
    accu = accuracy_score(yy[:batch_size*num_batches], pred)
    return f1*100, accu*100

def model(xx, yy, mask, train_cond=True):
    num_batches = int(len(xx)/batch_size)
    losses = 0
    preds = []
    for j in range(num_batches): 
        
        s = j * batch_size
        e = (j+1) * batch_size
        
        feed_dict = {x:xx[s:e], y:yy[s:e], x_mask:mask[s:e], is_train:train_cond, input_keep_prob:keep_prob, word_emb_mat:word_embedding}
        
        
        if(train_cond==True):
            _, _loss, step, _summary = sess.run([train_op, total_loss, global_step, summary], feed_dict)
            summary_writer.add_summary(_summary, step)  
#             print("Steps:{}".format(step), ", Loss: {}".format(_loss))

        else:
            _loss, pred = sess.run([total_loss, predictions], feed_dict)
            preds.append(pred)
            
        losses +=_loss

    if(train_cond==False):
        y_pred = []
        for i in range(num_batches):
            for pred in preds[i]:
                y_pred.append(pred)
        return losses/num_batches, y_pred
    
    return losses/num_batches, step

In [12]:
x_id_train, mask_train = data_prepare(x_train)
x_id_val, mask_val = data_prepare(x_val)
x_id_test, mask_test = data_prepare(x_test)
y_train = np.array(y_train)

In [13]:
num_epochs = 50

for i in range(num_epochs):
    
    random = np.random.choice(len(y_train), size=(len(y_train)), replace=False)
    x_id_train = x_id_train[random]
    y_train = y_train[random]
    mask_train = mask_train[random]
            
    losses, step = model(x_id_train, y_train, mask_train)
    print("Epoch:", i+1,"Step:", step, "loss:",losses)
    
    if((i+1)%5==0):
        saver.save(sess, save_path=save_dir)                         
        print("Saved Model Complete")
        train_loss, train_pred = model(x_id_train, y_train, mask_train, train_cond=False)
        f1_, accu_ = eval_score(y_train, train_pred)
        print("Train: F1 Score: ",  f1_, "Accuracy: ", accu_, "Loss: ", train_loss)
        val_loss, val_pred = model(x_id_val, y_val, mask_val, train_cond=False)
        f1_, accu_ = eval_score(y_val, val_pred)
        print("Val: F1 Score: ",  f1_, "Accuracy: ", accu_, "Loss: ", val_loss)
        
test_loss, test_pred = model(x_id_test, y_test, mask_test, train_cond=False)
f1_, accu_ = eval_score(y_test, test_pred)
print("Test: F1 Score: ",  f1_, "Accuracy: ", accu_, "Loss: ", test_loss)

Epoch: 1 Step: 48 loss: 3.91447790464
Epoch: 2 Step: 96 loss: 0.884176868945
Epoch: 3 Step: 144 loss: 0.723947924872
Epoch: 4 Step: 192 loss: 0.664846637597
Epoch: 5 Step: 240 loss: 0.67226972493
Saved Model Complete
Train: F1 Score:  57.4369328488 Accuracy:  85.15625 Loss:  0.594069524979
Val: F1 Score:  57.991087344 Accuracy:  86.875 Loss:  0.563192546368
Epoch: 6 Step: 288 loss: 0.594894857456
Epoch: 7 Step: 336 loss: 0.557087525725
Epoch: 8 Step: 384 loss: 0.579419625923
Epoch: 9 Step: 432 loss: 0.555498012652
Epoch: 10 Step: 480 loss: 0.54021554254
Saved Model Complete
Train: F1 Score:  82.8064308527 Accuracy:  91.9921875 Loss:  0.520881655316
Val: F1 Score:  80.1755418865 Accuracy:  90.9375 Loss:  0.535224729776
Epoch: 11 Step: 528 loss: 0.522731513406
Epoch: 12 Step: 576 loss: 0.540709823991
Epoch: 13 Step: 624 loss: 0.517984733606
Epoch: 14 Step: 672 loss: 0.521386946241
Epoch: 15 Step: 720 loss: 0.509815765545
Saved Model Complete
Train: F1 Score:  80.7915541389 Accuracy:  91.

In [ ]:
# num_epochs = 6
# for i in range(num_epochs):
#     random = np.random.choice(len(y_train), size=(len(y_train)), replace=False)
#     x_id_train = x_id_train[random]
#     y_train = y_train[random]
#     mask_train = mask_train[random]
            
#     losses, step = model(x_id_train, y_train, mask_train)
#     print("Epoch:", i+1,"Step:", step, "loss:",losses)
#     saver.save(sess, save_path=save_dir)                         
#     print("Saved Model Complete")
    
#     if((i+1)%2==0):
#         train_loss, train_pred = model(x_id_train, y_train, mask_train, train_cond=False)
#         f1_, accu_ = eval_score(y_train, train_pred)
#         print("Train: F1 Score: ",  f1_, "Accuracy: ", accu_, "Loss: ", train_loss)
#         val_loss, val_pred = model(x_id_val, y_val, mask_val, train_cond=False)
#         f1_, accu_ = eval_score(y_val, val_pred)
#         print("Val: F1 Score: ",  f1_, "Accuracy: ", accu_, "Loss: ", val_loss)
        
# test_loss, test_pred = model(x_id_test, y_test, mask_test, train_cond=False)
# f1_, accu_ = eval_score(y_test, test_pred)
# print("Test: F1 Score: ",  f1_, "Accuracy: ", accu_, "Loss: ", test_loss)

In [ ]:
# saver.restore(sess, save_dir)